## DD Utilities

In [ ]:
import numpy as np

In [ ]:
def Linear_system(n,sread,iparam):
    if sread == False:
        np.random.seed(1)
        M= n*np.identity(n) + np.random.rand(n,n) #uses uniform dist on [0,1)
        y= np.random.rand(n,1)
    else:
        import csv
        with open('iparam' + str(iparam) + '.csv') as file:
            csv_reader = csv.reader(file)
            data = [row for row in csv_reader]
            
            n= int(data[0][0])
            print(data[1])
            p= int(data[1][0])
            K= int(data[2][0])
            alpha= float(data[3][0])
            nn= np.array([int(d) for d in data[4]])
            nb= np.array([int(d) for d in data[5]])
            ne= np.array([int(d) for d in data[6]])
            
            M= np.zeros((n,n))
            y= np.zeros((n,1))
            for i in range(n):
                M[i,:]= data[10 + i]
            for i in range(n):
                y[i,:]= data[10 + n + i]
    
    x= np.linalg.solve(M,y)
    xstore= x
    
    return M,x,y,xstore

In [ ]:
def save_to_csv(n,p,K,alpha,nb,nn,ne,M,y,mpert,wpert,iparam):
    import csv
    f = open('iparam' + str(iparam) + '.csv', 'w', newline='')
    writer = csv.writer(f)
    writer.writerow([n])
    writer.writerow([p])
    writer.writerow([K])
    writer.writerow([alpha])
    writer.writerow(nn)#row5
    writer.writerow(nb)
    writer.writerow(ne)
    writer.writerow(nn)#nnb
    writer.writerow(nb)#nnt
    writer.writerow(ne)#nne
    writer.writerows(M)
    writer.writerows(y)
    writer.writerow([mpert])
#     f.close()
    
#     f = open('svpert'+str(iparam)+'.csv', 'w')
#     writer = csv.writer(f)
    writer.writerows(wpert)
    f.close()

In [ ]:
def Make_wpert(n,p,K,sread,iparam):
    if sread == False:
        wpert=np.random.randn(n*p*K,1)
    else:
        import csv
        with open('iparam' + str(iparam) + '.csv') as file:
            csv_reader = csv.reader(file)
            data = [row for row in csv_reader]
            wpert= np.zeros((n*p*K,1))
            for i in range(n*p*K):
                wpert[i,:]= float(data[i][0])
                
    return wpert

In [ ]:
def fix_domains(n,p,b,e,iprint):
    #creates nn, ne, nb, and index which are needed later
    
    import math
    if type(e) == int:
        overlap= e
        e= []
        for i in range(p-1):
            e+=[b[i + 1] + overlap - 1]
            if e[-1] > n: #will reset any of them to be <= n
                e[-1]= n
        e+=[n] #because the last end must be n

    nb= np.array(b)
    ne= np.array(e)
    nn= ne - nb + 1
    nnmax= np.max(nn)

    index=n p.zeros((p,nnmax))
    for i in range(p):
        index[i,0:nn[i]]= np.linspace(nb[i],ne[i],nn[i])

    return nn, nb, ne, index

In [ ]:
def Rmatrices(n,p,nn,index,iprint):
    nnmax=np.max(nn)
    R=np.zeros((p,nnmax,n))
    
    for i in range(0,p):
        for j in range(0,nn[i]):
            R[i,j,int(index[i,j] - 1)]= 1 
    
    return R

In [ ]:
def Amatrices(M,R,n,p,nn,iprint):
    nnmax= np.max(nn)
    A= np.zeros((p,nnmax,nnmax))
    RR= np.zeros((nnmax,n))
    AA= np.zeros((nnmax,nnmax))
    
    for i in range(0,p):
        RR= R[i] 
        AA= RR @ M @ RR.T
        A[i]= AA
        
    return A

In [ ]:
def Bmatrices(A, R, n, p, nn, iprint):
    nnmax= max(nn)

    B= np.zeros((p,n,n))
    RR= np.zeros((nnmax,n))
    AA= np.zeros((nnmax,nnmax))
    
    for i in range(0,p):
        RR= R[i]
        AA= A[i]
        B[i]= RR[0:nn[i],:].T @ np.linalg.inv(AA[0:nn[i],0:nn[i]]) @ RR[0:nn[i],:]
    
    if iprint >= 6:
        print('Print B matrices')
        for i in range(p):
            tmp= np.zeros((n,n))
            print('i=',i)
            tmp[0:n,0:n]= B[i,0:n,0:n]
            tmp[1:n,1:n]
        
    return B

In [ ]:
def Cmatrices(M,B,n,p,iprint):
    C= np.zeros((p,n,n))
    CC= np.zeros((n,n))
    BB= np.zeros((n,n))
    for i in range(p):
        BB= B[i]
        CC= BB @ M
        C[i]= CC

    if iprint >= 6:
        print('Print C matrices ')
        for i in range(p):
            tmp= np.zeros((n,n))
            print('Subdomain',i)
            tmp[1:n,1:n]= C[i,1:n,1:n]
            print(tmp[1:n,1:n])

    return C

In [1]:
def fvector(B,y,n,p,iprint):
    # Construct f vector for either additive or multiplicative Schwarz
    
    f= np.zeros((p,n))
    BB= np.zeros((n,n))
    for i in range(p):
        BB= B[i]
        f[i]= BB @ y[0:n,0]
    if iprint >= 4:
        print("Print f vector ")
        for i in range (p):
            tmp= np.zeros((n,1))
            print("i=",i)
            tmp[0:n,0]= f[i,0:n]
            print(tmp[0:n,0])
            
    return f

In [ ]:
def solution_compare(xstore,vstore,wstore,zstore,iprint):

    nx= len(xstore)
    nv= len(vstore)

    print(f'norm(exact - iterative) solutions    = {np.linalg.norm(xstore[0:nx,0] - vstore[nv - nx:nv,0]):13.6e}') 

    if iprint >= 2:
        print(f'Difference between vstore and wstore = {np.linalg.norm(vstore - wstore):13.6e}')
        print(f'Difference between wstore and zstore = {np.linalg.norm(wstore - zstore):13.6e}')
        print(f'Difference between vstore and zstore = {np.linalg.norm(vstore - zstore):13.6e}')

    if iprint >= 6:
        print(xstore[0:nx,0] - vstore[nv - nx:nv,0])
        print(vstore - wstore)
        print(wstore - zstore)
        print(vstore - zstore)